In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
import os
import pandas as pd

# Review subdirectories to find the most relevant match for Lung Cancer
all_dirs = os.listdir(tcga_root_dir)

# Since our trait is "Lung_Cancer", we look for directories containing "Lung" in the name
lung_related_dirs = [d for d in all_dirs if "Lung" in d]

# Choose the most specific directory for our task
# For Lung Cancer, we have multiple options: TCGA_Lung_Cancer_(LUNG), TCGA_Lung_Adenocarcinoma_(LUAD), 
# and TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)
# TCGA_Lung_Cancer_(LUNG) is the most general and appropriate for our trait
selected_dir = "TCGA_Lung_Cancer_(LUNG)"
selected_path = os.path.join(tcga_root_dir, selected_dir)

# Get paths to the clinical and genetic data files
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)

# Load the data files
clinical_data = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_data = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# Print the column names of the clinical data
print("Clinical data columns:")
print(clinical_data.columns.tolist())

# Also print basic information about both datasets
print("\nClinical data shape:", clinical_data.shape)
print("Genetic data shape:", genetic_data.shape)


Clinical data columns:
['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_0_no_1_yes', '_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplas

### Step 2: Find Candidate Demographic Features

In [3]:
import os

# Check the contents of the TCGA root directory to identify the correct folder
print(f"Checking contents of {tcga_root_dir}")
tcga_folders = os.listdir(tcga_root_dir)
print(f"Available folders: {tcga_folders}")

# Find a folder that might contain lung cancer data
lung_folder = None
for folder in tcga_folders:
    if "lung" in folder.lower():
        lung_folder = folder
        break

# If no lung-specific folder found, use the first available folder
if not lung_folder and tcga_folders:
    lung_folder = tcga_folders[0]
    
if lung_folder:
    print(f"Using folder: {lung_folder}")
    cohort_dir = os.path.join(tcga_root_dir, lung_folder)
    
    # Identify candidate columns for age and gender
    candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']
    candidate_gender_cols = ['gender']
    
    # Load the clinical data
    clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    
    # Extract and preview age-related columns
    age_preview = {}
    if candidate_age_cols:
        available_age_cols = [col for col in candidate_age_cols if col in clinical_df.columns]
        if available_age_cols:
            age_df = clinical_df[available_age_cols]
            age_preview = preview_df(age_df)
            print("Age-related columns preview:")
            print(age_preview)
        else:
            print("None of the candidate age columns are available in the dataset")
    
    # Extract and preview gender-related columns
    gender_preview = {}
    if candidate_gender_cols:
        available_gender_cols = [col for col in candidate_gender_cols if col in clinical_df.columns]
        if available_gender_cols:
            gender_df = clinical_df[available_gender_cols]
            gender_preview = preview_df(gender_df)
            print("Gender-related columns preview:")
            print(gender_preview)
        else:
            print("None of the candidate gender columns are available in the dataset")
else:
    print(f"No folders found in {tcga_root_dir}")


Checking contents of ../../input/TCGA
Available folders: ['TCGA_Liver_Cancer_(LIHC)', 'TCGA_Lower_Grade_Glioma_(LGG)', 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)', 'TCGA_Lung_Adenocarcinoma_(LUAD)', 'TCGA_Lung_Cancer_(LUNG)', 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)', 'TCGA_Melanoma_(SKCM)', 'TCGA_Mesothelioma_(MESO)', 'TCGA_Ocular_melanomas_(UVM)', 'TCGA_Ovarian_Cancer_(OV)', 'TCGA_Pancreatic_Cancer_(PAAD)', 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)', 'TCGA_Prostate_Cancer_(PRAD)', 'TCGA_Rectal_Cancer_(READ)', 'TCGA_Sarcoma_(SARC)', 'TCGA_Stomach_Cancer_(STAD)', 'TCGA_Testicular_Cancer_(TGCT)', 'TCGA_Thymoma_(THYM)', 'TCGA_Thyroid_Cancer_(THCA)', 'TCGA_Uterine_Carcinosarcoma_(UCS)', '.DS_Store', 'CrawlData.ipynb', 'TCGA_Acute_Myeloid_Leukemia_(LAML)', 'TCGA_Adrenocortical_Cancer_(ACC)', 'TCGA_Bile_Duct_Cancer_(CHOL)', 'TCGA_Bladder_Cancer_(BLCA)', 'TCGA_Breast_Cancer_(BRCA)', 'TCGA_Cervical_Cancer_(CESC)', 'TCGA_Colon_and_Rectal_Cancer_(COADREAD)', 'TCGA_Colon_Cancer_(COAD)

### Step 3: Select Demographic Features

In [4]:
# Select the appropriate columns for age and gender
age_col = "age_at_initial_pathologic_diagnosis"
gender_col = "gender"

# Print information about the chosen columns
print(f"Selected age column: {age_col}")
print(f"Age column sample values: {['67.0', '67.0', '72.0', '72.0', '77.0']}")
print(f"Selected gender column: {gender_col}")
print(f"Gender column sample values: {['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}")


Selected age column: age_at_initial_pathologic_diagnosis
Age column sample values: ['67.0', '67.0', '72.0', '72.0', '77.0']
Selected gender column: gender
Gender column sample values: ['MALE', 'MALE', 'MALE', 'MALE', 'FEMALE']


### Step 4: Feature Engineering and Validation

In [5]:
# Let's first get the correct folder for Lung Cancer data
selected_dir = "TCGA_Lung_Cancer_(LUNG)"
selected_path = os.path.join(tcga_root_dir, selected_dir)

# Reload the clinical and genetic data files to ensure we're using the correct dataset
clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(selected_path)
clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
genetic_df = pd.read_csv(genetic_file_path, index_col=0, sep='\t')

# 1. Extract and standardize clinical features (trait, age, gender)
selected_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# 2. Normalize gene symbols in gene expression data
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")
print(f"Gene data shape after normalization: {normalized_gene_df.shape}")

# 3. Link clinical and genetic data
# Transpose the genetic data to have samples as rows
genetic_df_t = normalized_gene_df.T
# Ensure the indices match between datasets
common_samples = list(set(genetic_df_t.index) & set(selected_clinical_df.index))
genetic_df_filtered = genetic_df_t.loc[common_samples]
clinical_df_filtered = selected_clinical_df.loc[common_samples]

# Combine the datasets
linked_data = pd.concat([clinical_df_filtered, genetic_df_filtered], axis=1)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data_cleaned.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data_filtered = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate data quality and save cohort information
# First check if gene and trait data are available
is_gene_available = linked_data_filtered.shape[1] > 3  # More columns than just trait, age, gender
is_trait_available = trait in linked_data_filtered.columns

# Second validation for saving metadata
note = f"Dataset contains {linked_data_filtered.shape[0]} samples and {linked_data_filtered.shape[1] - 3} genes after preprocessing."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_filtered,
    note=note
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_filtered.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
    
    # Also save the clinical data separately
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_cols = [col for col in linked_data_filtered.columns if col in [trait, 'Age', 'Gender']]
    linked_data_filtered[clinical_cols].to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Dataset not usable for analysis. Data not saved.")

Normalized gene data saved to ../../output/preprocess/Lung_Cancer/gene_data/TCGA.csv
Gene data shape after normalization: (19848, 1129)
Linked data shape: (1129, 19851)


Data shape after handling missing values: (1129, 19851)
For the feature 'Lung_Cancer', the least common label is '0' with 110 occurrences. This represents 9.74% of the dataset.
The distribution of the feature 'Lung_Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 455 occurrences. This represents 40.30% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Lung_Cancer/TCGA.csv
Clinical data saved to ../../output/preprocess/Lung_Cancer/clinical_data/TCGA.csv
